<a href="https://colab.research.google.com/github/karolinablaszka/Project-ML/blob/EDA/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.covariance import EllipticEnvelope

In [16]:
df = pd.read_csv(r'C:/Users/karol/data_ML/train_data.csv', header=None)

TypeError: read_csv() got an unexpected keyword argument 'on_bad_lines'

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,304.361,-436.590,-33854.769,-96571.569,-15086.947,-794.409,-442.859,304.248,-202.411,-506.852,...,95.294,-3.936,-251.812,256.429,-473.661,-1398.604,42.638,771.185,-191.123,1356.137
1,54.047,90.087,-154.747,-4116.486,38365.133,-589.309,1115.367,274.199,814.953,167.272,...,-176.341,-426.238,-542.428,99.147,-102.309,-277.948,-98.859,-1163.779,265.231,-992.056
2,272.168,-201.736,4212.592,-9123.655,-35422.458,-1029.249,-506.290,-476.856,-570.731,-161.288,...,321.426,-376.033,142.834,920.958,-122.525,-174.304,-137.612,-1571.473,678.323,1020.263
3,170.007,68.736,48662.079,29735.235,-13903.955,895.081,-257.748,-811.058,-691.561,-31.439,...,-409.919,400.946,313.270,123.172,1786.962,147.637,31.433,-784.719,573.433,1454.415
4,86.797,180.052,-49643.545,2515.406,26206.315,-407.453,-189.416,-53.664,-159.507,-42.291,...,-101.761,-424.898,37.254,-337.431,423.691,14.240,267.352,-234.560,-213.804,873.391


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750 entries, 0 to 3749
Columns: 10000 entries, 0 to 9999
dtypes: float64(10000)
memory usage: 286.1 MB


In [7]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
count,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000,...,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000,3750.000000
mean,30.644135,9.080955,169.679419,811.083532,-186.008805,25.717789,24.918118,19.061259,14.826002,14.970447,...,13.997377,21.456582,22.586461,16.736889,13.992185,-1.426345,9.597250,-2.031243,-2.841237,38.071903
std,286.172414,213.166659,43665.003613,57123.457110,21942.466227,711.556331,898.109174,868.676044,532.727247,236.749501,...,269.223339,545.746530,265.221187,503.416444,818.825405,655.666258,204.731554,793.022407,629.128655,768.405732
min,-989.450000,-859.922000,-163336.428000,-243844.632000,-77566.109000,-2414.196000,-3227.681000,-3151.321000,-1792.781000,-794.271000,...,-957.937000,-1983.009000,-937.357000,-1758.858000,-2526.791000,-2165.769000,-739.205000,-2731.037000,-2223.909000,-2565.513000
25%,-163.781750,-135.343500,-29284.119500,-38028.429000,-14426.799500,-448.696750,-590.170250,-562.956500,-349.689750,-144.157750,...,-169.777250,-352.959500,-154.639750,-328.083750,-550.599000,-432.000750,-129.076250,-561.304000,-424.192750,-462.828000
50%,35.478500,2.680500,1181.698500,980.588000,-154.835000,13.931500,13.674000,16.750500,9.731000,11.159500,...,18.700000,38.348500,26.199500,14.575500,1.363500,-3.796500,8.600500,-4.243000,-7.628000,28.107000
75%,215.780500,156.095750,29630.914500,40561.960250,14515.833000,506.893250,630.490750,608.698000,383.110500,179.768250,...,196.049000,396.868500,198.237250,356.185500,574.453250,425.338000,149.855750,539.550000,427.794500,552.173500
max,1224.394000,722.868000,177744.241000,215372.854000,86287.955000,2446.466000,3118.087000,2966.971000,1914.308000,803.200000,...,893.196000,2080.750000,1070.453000,1705.664000,2868.989000,2562.331000,685.176000,3123.617000,2204.527000,2664.074000


In [8]:
df.isnull().sum().sum() 

0

In [9]:
df.isnull().sum().sort_values()

0       0
6663    0
6664    0
6665    0
6666    0
       ..
3334    0
3335    0
3336    0
3329    0
9999    0
Length: 10000, dtype: int64

In [10]:
df.dropna(axis=1, how='any', inplace=True)

In [11]:
df.drop_duplicates(subset=None, keep='first', inplace=True)

In [12]:
df.shape

(3750, 10000)

In [13]:
df.nunique()

0       3742
1       3735
2       3750
3       3750
4       3750
        ... 
9995    3746
9996    3745
9997    3749
9998    3746
9999    3748
Length: 10000, dtype: int64

In [14]:
corr_matrix = df.corr().abs()
print("corr matrix", corr_matrix)

corr matrix           0         1         2         3         4         5         6     \
0     1.000000  0.009717  0.007823  0.010449  0.023684  0.007333  0.015694   
1     0.009717  1.000000  0.014160  0.023034  0.003338  0.001651  0.001523   
2     0.007823  0.014160  1.000000  0.010302  0.015079  0.018820  0.012610   
3     0.010449  0.023034  0.010302  1.000000  0.024708  0.000070  0.007635   
4     0.023684  0.003338  0.015079  0.024708  1.000000  0.017415  0.018771   
...        ...       ...       ...       ...       ...       ...       ...   
9995  0.005980  0.017223  0.025339  0.006347  0.019803  0.022484  0.001115   
9996  0.004817  0.004058  0.032476  0.013508  0.004072  0.020380  0.004446   
9997  0.002374  0.002938  0.018478  0.010463  0.029769  0.012821  0.003756   
9998  0.004539  0.012621  0.017792  0.003039  0.000157  0.008822  0.003015   
9999  0.037154  0.016903  0.021163  0.015791  0.005128  0.017924  0.010467   

          7         8         9     ...      9990  

In [15]:
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(corr_matrix, k=1).astype(np.bool))
print("upper triangle", upper)

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
to_drop

# Drop features
df.drop(df.columns[to_drop], axis=1)

upper triangle       0         1         2         3         4         5         6     \
0      NaN  0.009717  0.007823  0.010449  0.023684  0.007333  0.015694   
1      NaN       NaN  0.014160  0.023034  0.003338  0.001651  0.001523   
2      NaN       NaN       NaN  0.010302  0.015079  0.018820  0.012610   
3      NaN       NaN       NaN       NaN  0.024708  0.000070  0.007635   
4      NaN       NaN       NaN       NaN       NaN  0.017415  0.018771   
...    ...       ...       ...       ...       ...       ...       ...   
9995   NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9996   NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9997   NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9998   NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9999   NaN       NaN       NaN       NaN       NaN       NaN       NaN   

          7         8         9     ...      9990      9991      9992  \
0     0.013918  0.01878

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,304.361,-436.590,-33854.769,-96571.569,-15086.947,-794.409,-442.859,304.248,-202.411,-506.852,...,95.294,-3.936,-251.812,256.429,-473.661,-1398.604,42.638,771.185,-191.123,1356.137
1,54.047,90.087,-154.747,-4116.486,38365.133,-589.309,1115.367,274.199,814.953,167.272,...,-176.341,-426.238,-542.428,99.147,-102.309,-277.948,-98.859,-1163.779,265.231,-992.056
2,272.168,-201.736,4212.592,-9123.655,-35422.458,-1029.249,-506.290,-476.856,-570.731,-161.288,...,321.426,-376.033,142.834,920.958,-122.525,-174.304,-137.612,-1571.473,678.323,1020.263
3,170.007,68.736,48662.079,29735.235,-13903.955,895.081,-257.748,-811.058,-691.561,-31.439,...,-409.919,400.946,313.270,123.172,1786.962,147.637,31.433,-784.719,573.433,1454.415
4,86.797,180.052,-49643.545,2515.406,26206.315,-407.453,-189.416,-53.664,-159.507,-42.291,...,-101.761,-424.898,37.254,-337.431,423.691,14.240,267.352,-234.560,-213.804,873.391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3745,338.539,-98.216,-37488.883,-2346.403,-291.325,553.155,1041.511,391.664,1016.730,49.772,...,350.501,-607.873,430.407,-469.737,68.670,328.636,105.448,-368.834,378.017,-253.044
3746,105.511,-167.468,54447.468,-38055.924,-12394.035,711.218,100.459,-1377.005,-171.175,-325.444,...,4.759,-9.079,104.333,4.676,-1069.879,-92.252,86.110,-4.935,433.674,-355.118
3747,93.186,-155.288,-16555.534,-10605.421,-3027.792,-80.498,-2201.598,-49.392,-848.946,-252.714,...,-342.855,-151.956,7.386,564.910,613.541,-570.735,-5.810,797.659,-289.170,-2025.470
3748,857.066,98.182,-39429.721,26215.357,-1465.352,-340.496,-992.109,990.646,140.943,-281.100,...,200.827,-1445.572,-39.879,421.075,-191.389,-538.616,141.365,552.974,310.130,207.084


MemoryError: Unable to allocate 763. MiB for an array with shape (100000000,) and data type int64

MemoryError: Unable to allocate 763. MiB for an array with shape (100000000,) and data type float64

<Figure size 432x288 with 2 Axes>

In [ ]:
outlier_detector = EllipticEnvelope(contamination=.1)

outlier_detector.fit(df)

outlier_detector.predict(df)